In [2]:
from decouple import config

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None 

In [4]:
ALPHA_VANTAGE_API_KEY=config("ALPHA_VANTAGE_API_KEY", default=None, cast=str)

In [30]:
import requests
api_key=ALPHA_VANTAGE_API_KEY
params={
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
    "function": "TIME_SERIES_INTRADAY"
}
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function={function}&symbol={ticker}&interval=1min&apikey={api_key}'.format(**params)
r = requests.get(url)
data = r.json()


In [36]:
dataset_key=[x for x in list(data.keys()) if not x.lower()=="meta data"][0]

In [38]:
results=data[dataset_key]

In [39]:
timestamp_str=list(results.keys())[0]
timestamp_str

'2025-04-17 19:59:00'

In [40]:
result_val=results.get(timestamp_str)
result_val

{'1. open': '197.2500',
 '2. high': '197.3200',
 '3. low': '197.2400',
 '4. close': '197.3000',
 '5. volume': '847'}

In [41]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str,result):
    # unix_timestamp = result.get('t') / 1000.0
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S' 
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str,timestamp_format)).astimezone(utc)
    return {
        'open_price': Decimal(result['1. open']),
        'close_price': Decimal(result['4. close']),
        'high_price': Decimal(result['2. high']),
        'low_price': Decimal(result['3. low']),
        'number_of_trades': None,
        'volume': int(result['5. volume']),
        'volume_weighted_average': None,
        'time': timestamp,
    }

In [42]:
transform_alpha_vantage_result(timestamp_str,result_val)

{'open_price': Decimal('197.2500'),
 'close_price': Decimal('197.3000'),
 'high_price': Decimal('197.3200'),
 'low_price': Decimal('197.2400'),
 'number_of_trades': None,
 'volume': 847,
 'volume_weighted_average': None,
 'time': datetime.datetime(2025, 4, 17, 23, 59, tzinfo=<UTC>)}